In [ ]:
import os,cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils import np_utils
import tensorflow
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten, Deconvolution2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD,RMSprop,adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#<----------------------Load Train Images--------------------->
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
data_path='C:/Users/admin/Documents/CMATERdb/C_full/Train'
data_dir_list = os.listdir(data_path)
img_rows=28
img_cols=28
num_channel=1
num_classes = 238

labels_name={}
for i in range(num_classes):
    #labels_name[str(i)]=i
    labels_name[str(i+1)]=i
print(labels_name)
img_data_list=[]
labels_list = []
for dataset in data_dir_list:
	img_list=os.listdir(data_path+'/'+ dataset)
	print ('Loading the images of dataset-'+'{}\n'.format(dataset))
	label = labels_name[dataset]
	print(label)
	for img in img_list:
		input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
		input_img=cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
		input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
		img_data_list.append(input_img_resize)
		labels_list.append(label)
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

labels = np.array(labels_list)
print(np.unique(labels,return_counts=True))
Y = np_utils.to_categorical(labels, num_classes)


x,y = shuffle(img_data,Y, random_state=2)

X_train=x
y_train=y

if K.image_data_format() == 'channels_first':
    x_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

print('input_shape_train: ', input_shape)

In [ ]:
#<----------------------Load Test-Validation Images / Cross Test & Validation Images--------------------->
data_path2 ='C:/Users/admin/Documents/CMATERdb/C_full/Test'
data_dir_list2 = os.listdir(data_path2)
img_rows2=28
img_cols2=28

num_classes2 = 238

labels_name2={}
for i in range(num_classes2):
    #labels_name2[str(i)]=i
    labels_name2[str(i+1)]=i

print(labels_name2)
img_data_list2=[]
labels_list2 = []
for dataset2 in data_dir_list2:
	img_list2=os.listdir(data_path2+'/'+ dataset2)
	print ('Loading the images of dataset-'+'{}\n'.format(dataset2))
	label2 = labels_name2[dataset2]
	print(label2)
	for img2 in img_list2:
		input_img2=cv2.imread(data_path2 + '/'+ dataset2 + '/'+ img2 )
		input_img2=cv2.cvtColor(input_img2,cv2.COLOR_BGR2GRAY)
		input_img_resize2=cv2.resize(input_img2,(img_rows2,img_cols2))
		img_data_list2.append(input_img_resize2)
		labels_list2.append(label2)
img_data2 = np.array(img_data_list2)
img_data2 = img_data2.astype('float32')
img_data2 /= 255
print (img_data2.shape)

labels2 = np.array(labels_list2)
print(np.unique(labels2,return_counts=True))
Y2 = np_utils.to_categorical(labels2, num_classes2)

x2,y2 = shuffle(img_data2,Y2, random_state=2)
X_vali2, X_test2, y_vali2, y_test2 = train_test_split(x2, y2, test_size=0.5, random_state=2)
if K.image_data_format() == 'channels_first':
    x_test2 = X_test2.reshape(X_test2.shape[0], 1, img_rows2, img_cols2)
    input_shape2 = (1, img_rows2, img_cols2)
else:
    x_test2 = X_test2.reshape(X_test2.shape[0], img_rows2, img_cols2, 1)
    input_shape2 = (img_rows2, img_cols2, 1)

print('input_shape_test_vali: ', input_shape2)

In [ ]:
w, h = img_rows, img_cols
input_img = Input(shape = (w, h, num_channel))
#<----------------------DConvAENNet PART 1: UNSUPERVISED LEARNING (ENCODER)--------------------->
x = Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.35)(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
Encoder = MaxPooling2D((2, 2), padding='same')(x)

#<-----------------------DConvAENNet PART 2: UNSUPERVISED LEARNING (DECODER)---------------------->
x = Conv2D(16, (3, 3), activation='relu', padding='same')(Encoder)
x = Dropout(0.35)(x)
x = Conv2D(64, (3, 3),activation='relu', padding='same')(x)
x = Conv2D(128 , (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((7, 7))(x)
Decoder = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

Auto_Encoder = Model(input_img, Decoder)
Auto_Encoder.compile(optimizer='RMSprop', loss='binary_crossentropy',metrics=['accuracy'])
Auto_Encoder.summary()

In [ ]:
X_train = X_train.reshape(-1,img_rows, img_cols, 1)   
X_vali2 = X_vali2.reshape(-1,img_rows, img_cols, 1)   
X_test2 = X_test2.reshape(-1,img_rows, img_cols, 1)

In [ ]:
Datagen = ImageDataGenerator( rotation_range=0.2, width_shift_range=0.2, height_shift_range=0.2 )
Datagen.fit(X_train,augment=True)
Auto_Encoder.fit_generator(Datagen.flow(X_train, X_train, batch_size=112,shuffle=True), epochs=20)

In [ ]:
#<---------------------Augmented Images-------------------------->
import matplotlib.pyplot as plt
for X_batch, y_batch in Datagen.flow(X_train, y_train, batch_size=112):
	for i in range(0, 9):
		plt.subplot(330 + 1 + i)
		plt.imshow(X_batch[i].reshape(28, 28), cmap=plt.get_cmap('gray'))
	plt.show()
	break

In [ ]:
#<--------------------DConvAENNet PART 3: SUPERVISED LEARNING (ENCODER AND FULLY CONNECTED DENSE LAYERS)------------------>
x=Flatten()(Encoder)
x=Dense(3*num_classes,activation='relu')(x)                
x=Dense(num_classes,activation='softmax')(x)
MODEL=Model([input_img],[x])
MODEL.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
MODEL.summary()
super_train_aug = ImageDataGenerator(rotation_range=0.0,width_shift_range=0.0,height_shift_range=0.0)
super_train_aug.fit(X_train, augment=True)
super_test_aug = ImageDataGenerator()
super_test_aug.fit(X_test2, augment=True)
super_train = super_train_aug.flow(X_train, y_train,batch_size=112)                   
super_vali = super_test_aug.flow(X_vali2, y_vali2,batch_size=112)

In [ ]:
history=MODEL.fit_generator(super_train,epochs=50,validation_data=super_vali,shuffle=True)

In [ ]:
score = MODEL.evaluate(X_test2,y_test2, verbose=0)
print("DConvAENNet Testing Accuracy: %.2f%%" % (score[1]*100))
print("DConvAENNet Testing Error: %.2f%%" % (100-score[1]*100))

In [ ]:
print("Keys: ")
print(history.history.keys())
print("Train Accuracy: ")
print(history.history['accuracy'])
print("Val. Accuracy: ")
print(history.history['val_accuracy'])

print("Train Loss: ")
print(history.history['loss'])
print("Val. Loss: ")
print(history.history['val_loss'])

In [ ]:
#<--------------------Classification Report-------------------->
from sklearn.metrics import classification_report
Y_test = np.argmax(y_test2, axis=1)
y_pred = MODEL.predict(X_test2)
print(classification_report(Y_test, np.argmax(y_pred, axis = 1)))

In [ ]:
#<--------------------Generate Confusion Matrix-------------------->
import sklearn
matrix = sklearn.metrics.confusion_matrix(Y_test, np.argmax(y_pred, axis = 1))
print(matrix)
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
print(np.array(matrix))

In [ ]:
#<--------------------Plot Confusion Matrix -------------------->
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt


df_cm = pd.DataFrame(matrix, range(num_classes), range(num_classes))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 6})
plt.figure(figsize=(7,7))
plt.show()

In [ ]:
#<-------------------- serialize model to JSON ---------------------->
model_json = MODEL.to_json()
with open("model_name.json", "w") as json_file:
	json_file.write(model_json)
#<-------------------- serialize weights to HDF5 ---------------------->
MODEL.save_weights("model_name.h5")
print("Saved model to disk")

In [ ]:
#<-------------------- Plot histroy -------------------->
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy vs Iteration')
plt.ylabel('Accuracy')
plt.xlabel('Iteration')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs Iteration')
plt.ylabel('Loss')
plt.xlabel('Iteration')
plt.legend(['train', 'test'], loc='upper right')
plt.show()